In [71]:
import sys
from matplotlib import rcParams 
rcParams["figure.figsize"] = (16,5)
sys.path.append('')

import librosa
import librosa.display

from TTS.tts.models.tacotron2 import Tacotron2 
from TTS.tts.utils import *
from TTS.tts.utils.generic_utils import setup_model
from TTS.utils.audio import AudioProcessor
from TTS.utils.io import load_config
from TTS.tts.utils.text import text_to_sequence, phoneme_to_sequence
from TTS.tts.utils.text.symbols import symbols, phonemes
import torch
from TTS.tts.utils.synthesis import synthesis

In [72]:
# Set constants

# MODEL_PATH = '/content/drive/MyDrive/Mestrado/TTS/GST_rosana_only/checkpoint_120000.pth.tar'
CONFIG_PATH =  './experiments/debug/config.json'

# MODEL_PATH ='/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/best_model.pth.tar'
# CONFIG_PATH = '/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/config.json'

CONFIG = load_config(CONFIG_PATH)
# CONFIG['datasets'][0]['path'] = './'
# CONFIG['output_path'] = './'
# CONFIG['audio']['signal_norm'] = False
# CONFIG['audio']['stats_path'] = ''
# CONFIG['use_phonemes'] = False
# CONFIG['save_step'] = 500
use_cuda = True

# load the model
ap = AudioProcessor(**CONFIG.audio)

num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)


# load the model
num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, 3, CONFIG, None)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:512
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:
 | > hop_length:128
 | > win_length:512
 > Using model: Tacotron2


In [73]:
model

Tacotron2(
  (speaker_embedding): Embedding(3, 512)
  (embedding): Embedding(217, 512, padding_idx=0)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0): ConvBNBlock(
        (convolution1d): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        (batch_normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (activation): ReLU()
      )
      (1): ConvBNBlock(
        (convolution1d): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        (batch_normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (activation): ReLU()
      )
      (2): ConvBNBlock(
        (convolution1d): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        (batch_normalization): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   

In [74]:
model.speaker_embedding

Embedding(3, 512)

In [75]:
size = (1, 3)
test = torch.rand(size).type(torch.long)
model.speaker_embedding(test).shape

torch.Size([1, 3, 512])

In [76]:
size = (1, 50, 80)
test = torch.rand(size).type(torch.float)
style_embed, logits = model.gst_layer(test)
print(style_embed.shape, logits.shape)

torch.Size([1, 1, 256]) torch.Size([1, 1, 1, 3])


In [77]:
logits.squeeze(0).squeeze(0).squeeze(0).detach().cpu().numpy()

array([0.33001867, 0.3326895 , 0.33729187], dtype=float32)

In [78]:
N = CONFIG['gst']['gst_style_tokens']

feats = np.zeros((6, N))
feats

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [79]:
feats[0] = logits.squeeze(0).squeeze(0).squeeze(0).detach().cpu().numpy()

In [80]:
feats[0:3]

array([[0.33001867, 0.33268949, 0.33729187],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [59]:
import os
def vctk_direct(root_path, meta_file):
    meta_path = os.path.join(root_path, meta_file)
    
    items = []
    
    with open(meta_path, 'r', encoding= 'utf-8') as f:
        for line in f:
            cols = line.split(',')
            if(cols[1] == 'text'): # The first element is header in my file
                continue
            wav_file = cols[0]
            text = cols[1]
            speaker_name = cols[2][:-1]  # The last char is "\n" since after this line is a breakline
            items.append([text, wav_file, speaker_name])
    return items

In [64]:
adriana = vctk_direct('./', 'adriana_expressivo_train.csv')

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 296-297: invalid continuation byte

In [66]:
import pandas as pd
df = pd.read_csv('adriana_expressivo_train.csv', encoding='latin-1', sep='|')

In [67]:
df.head()

,wav_path,text,emb_id
0,/l/disk1/awstebas/data/TTS/vozes_sync/Adriana/...,os leitores podem colaborar com esta rodada do...,3
1,/l/disk1/awstebas/data/TTS/vozes_sync/Adriana/...,e a notãâ­cia boa ãâ© que o raio x mostra qu...,3
2,/l/disk1/awstebas/data/TTS/vozes_sync/Adriana/...,ãâ impossãâ­vel nãâ£o gostar da histãâ³ri...,3
3,/l/disk1/awstebas/data/TTS/vozes_sync/Adriana/...,"se for ao reino da sibãâ©ria , busca ajuda da...",3
4,/l/disk1/awstebas/data/TTS/vozes_sync/Adriana/...,"jãâºnior , que investiga a guerra civil cario...",3


In [6]:
logits

tensor([[[[0.3378, 0.3444, 0.3178]],

         [[0.3369, 0.3411, 0.3220]],

         [[0.3310, 0.3294, 0.3396]],

         [[0.3223, 0.3372, 0.3405]]]], grad_fn=<SoftmaxBackward>)

In [7]:
text_cleaner = [CONFIG.text_cleaner]
text_cleaner

['basic_cleaners']

In [8]:
# s = '''Um professor, para se tornar doutor, investe, pelo menos, 8 anos de sua vida ( se partir da graduação direto pro doutorado). Se fizer graduação, mestrado e doutorado, são 10 anos. Se fizer graduação, especialização, mestrado e doutorado, lá se vão 12 anos. Sim, 12 anos de universidade. Todo esse tempo labutando com ensino, pesquisa e extensão. Produzindo, apresentando e publicando trabalhos.  Participando de debates e congressos. Dando aula. Ministrando cursos. Participando de bancas. Emitindo pareceres. Orientando artigos, projetos de conclusão de cursos, TCC's e etc, etc, etc. De repente, todo esse conhecimento arduamente produzido dentro dos parâmetros científicos e acadêmicos passa a não valer  nada ante a palavra de um esquizóide qualquer que define a forma como nós, professores, devemos conduzir o trabalho para o qual nos preparamos ao longo da vida e diuturnamente - pesquisando e estudando métodos e teorias, produzindo e submetendo nossas ideias à comunidade científica - que, todos sabemos, não é muito conhecida pela generosidade. Sem comentar os diversos perrengues -  familiares, socioemocionais, financeiros e de saúde -  que enfrentamos durante e  em virtude dessa caminhada extenuante e tão mal recompensada. Ainda assim, contra tudo e todos, persistimos cantando no mal tempo, porque temos a certeza daquilo que fazemos, e assentamos nossa autoridade e dignidade no conhecimento que produzimos. 
# Antes, a violência que se abatia sobre nós, profissionais da educação, era apenas financeira e a desvalorização social era decorrente dessa desvalorização financeira: '' Você é só uma professorazinha, ganha pouco''. 
# Agora, a violência é pior: a palavra e a autoridade intelectual do professor -  que ainda mantínhamos, apesar de tudo -  são jogados por terra. 
# Somos confrontados por pessoas que não leem, não têm conhecimento de absolutamente nada e que se informam apenas por youtubers, líderes religiosos delirantes ou figuras obscuras sem nenhum (re)conhecimento científico e que são  merecidamente proscritas do meio acadêmico.  Ainda por cima, somos chamados de '' doutrinadores'', acusados e vilipendiados das piores formas possíveis sem que os difamadores se retratem ou sofram as devidas punições por suas afirmações criminosas que têm como intuito deslegitimar, mais uma vez, a profissão docente nos violentando moralmente. E, como se não bastasse, há ''professores'' que se alinham a esse  evidente projeto de destruição da educação no país. Não me admira que a profissão esteja entre as menos procuradas pelos jovens, e que esteja, infelizmente, caminhando para a extinção.'''

import numpy as np

s = "ambulantes vendem os ingressos para a copa do mundo de dois mil e catorze , mas só alguns dias antes do dia do sorteio ."

# s = "e é"

seq = np.asarray(text_to_sequence(s, text_cleaner), dtype=np.int32)
chars_var = torch.from_numpy(seq).unsqueeze(0).cuda()

# style_wav = {'1': 0.3} # Parece um token interessante

# f = 0.1
# style_wav = {}
# n_tokens = 3
# for i in range(n_tokens):
#     style_wav[str(i)] = f

style_wav = './data/LJSpeech/LJSpeech-1.1/wavs/LJ001-0003.wav'

speaker_embedding = None
speaker_id = 2


# style_wav = None

use_cuda = True

wav, alignment, decoder_output, postnet_output, stop_tokens, _, logits = synthesis(
    model.cuda(),
    s,
    CONFIG,
    use_cuda,
    ap,
    speaker_id=speaker_id,
    speaker_embedding=speaker_embedding,
    style_wav=style_wav,
    truncated=False,
    enable_eos_bos_chars=CONFIG.enable_eos_bos_chars, #pylint: disable=unused-argument
    use_griffin_lim=True,
    do_trim_silence=False)

In [9]:
logits[0][0][0].detach().cpu()

tensor([0.2940, 0.3483, 0.3577])

In [10]:
logits.reshape(torch.tensor([0.0,0.0,1.0]).size())

tensor([0.2940, 0.3483, 0.3577], device='cuda:0')

In [13]:
chars_var.size()

torch.Size([1, 120])

In [32]:
torch.tensor([0.0,0.0,1.0])

tensor([0., 0., 1.])

In [29]:
from torch.nn import functional

In [34]:
loss = functional.binary_cross_entropy(logits.detach().cpu(), torch.tensor([0.0,0.0,1.0]))
loss

C:\Users\lucas\Anaconda3\envs\m_audio\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([1, 1, 1, 3])) is deprecated. Please ensure they have the same size.
  """Entry point for launching an IPython kernel.


tensor(0.6158)

In [35]:
alignment.shape

(8, 120)

In [16]:
type(logits)

NoneType

In [27]:
style_wav

'./data/LJSpeech/LJSpeech-1.1/wavs/LJ001-0003.wav'

In [ ]:
model.gst_layer.style_token_layer.style_tokens

In [30]:
def text_to_seqvec(text, CONFIG):
    text_cleaner = [CONFIG.text_cleaner]
    # text ot phonemes to sequence vector
    if CONFIG.use_phonemes:
        seq = np.asarray(
            phoneme_to_sequence(text, text_cleaner, CONFIG.phoneme_language,
                                CONFIG.enable_eos_bos_chars,
                                tp=CONFIG.characters if 'characters' in CONFIG.keys() else None),
            dtype=np.int32)
    else:
        seq = np.asarray(text_to_sequence(text, text_cleaner, tp=CONFIG.characters if 'characters' in CONFIG.keys() else None), dtype=np.int32)
    return seq


def numpy_to_torch(np_array, dtype, cuda=False):
    if np_array is None:
        return None
    tensor = torch.as_tensor(np_array, dtype=dtype)
    if cuda:
        return tensor.cuda()
    return tensor

def id_to_torch(speaker_id, cuda=False):
    if speaker_id is not None:
        speaker_id = np.asarray(speaker_id)
        speaker_id = torch.from_numpy(speaker_id).unsqueeze(0)
    if cuda:
        return speaker_id.cuda().type(torch.long)
    return speaker_id.type(torch.long)

def compute_style_mel(style_wav, ap, cuda=False):
    style_mel = torch.FloatTensor(ap.melspectrogram(
        ap.load_wav(style_wav, sr=ap.sample_rate))).unsqueeze(0)
    if cuda:
        return style_mel.cuda()
    return style_mel

In [15]:
speaker_id = id_to_torch(2, cuda=use_cuda)

In [19]:
inputs = text_to_seqvec(s, CONFIG)
inputs = numpy_to_torch(inputs, torch.long, cuda=use_cuda)
inputs = inputs.unsqueeze(0)

x = model.embedding(inputs).transpose(1, 2)
x = model.encoder.inference(x)

In [20]:
speaker_ids

tensor([2], device='cuda:0')

In [21]:
speaker_ids = speaker_id
gst = True
num_speakers = 3
style_mel = None

if gst:
    # B x gst_dim
    encoder_outputs, logits = model.compute_gst(x,
                                       style_mel,
                                       speaker_embeddings if model.gst_use_speaker_embedding else None)
if num_speakers > 1:
    if not model.embeddings_per_sample:
        speaker_embeddings = model.speaker_embedding(speaker_ids)[:, None]
    encoder_outputs = model._concat_speaker_embedding(encoder_outputs, speaker_embeddings)

In [22]:
inputs = text_to_seqvec(s, CONFIG)
inputs = numpy_to_torch(inputs, torch.long, cuda=use_cuda)
inputs = inputs.unsqueeze(0)

model.inference(text = inputs, speaker_ids = speaker_id, style_mel = None)

(tensor([[[-0.0305, -0.0450, -0.0005,  ...,  0.1477, -0.0312,  0.1719],
          [ 0.1300,  0.0743, -0.0043,  ...,  0.1124, -0.1079, -0.2240],
          [-0.1467,  0.3013,  0.0639,  ...,  0.0745, -0.0475, -0.0513],
          ...,
          [-0.2115,  0.2265, -0.1254,  ...,  0.0058,  0.1708,  0.0503],
          [ 0.0314, -0.0464,  0.0146,  ..., -0.0792,  0.3024, -0.1121],
          [ 0.0527, -0.1534, -0.0071,  ..., -0.0079, -0.2290,  0.1016]]],
        device='cuda:0'),
 tensor([[[-3.0516e-02,  8.6851e-01, -4.7641e-04,  ...,  1.4774e-01,
           -3.1177e-02,  1.7193e-01],
          [ 1.2995e-01,  7.4270e-02, -4.2638e-03,  ...,  1.1239e-01,
           -3.6288e+00, -2.2402e-01],
          [-4.6684e-01,  3.0128e-01,  6.3911e-02,  ..., -2.0052e+00,
            1.1703e+00,  1.1048e+00],
          ...,
          [-2.1146e-01,  2.2655e-01,  1.7077e+00,  ...,  5.7870e-03,
            1.7076e-01,  5.0268e-02],
          [ 3.1421e-02, -1.0175e-01,  1.4641e-02,  ...,  6.6859e-01,
            3

In [23]:
device = chars_var.device
device

device(type='cuda', index=0)

In [24]:
embedded_inputs = model.embedding(chars_var.type(torch.long)).transpose(1, 2)
encoder_outputs = model.encoder.inference(embedded_inputs)

In [25]:
encoder_outputs.shape

torch.Size([1, 120, 512])

In [26]:
query = torch.zeros(1, 1, model.gst_embedding_dim//2).to(device)
_GST = torch.tanh(model.gst_layer.style_token_layer.style_tokens)
gst_outputs = torch.zeros(1, 1, model.gst_embedding_dim).to(device)
for k_token, v_amplifier in style_wav.items():
    print(k_token, v_amplifier)
    key = _GST[int(k_token)].unsqueeze(0).expand(1, -1, -1)
    print(query.shape, key.shape)
    gst_outputs_att, _ = model.gst_layer.style_token_layer.attention(query, key)
    print(gst_outputs_att.shape)
    gst_outputs = gst_outputs + gst_outputs_att * v_amplifier
    
gst_outputs.shape

AttributeError: 'str' object has no attribute 'items'

In [ ]:
encoder_outputs = model._concat_speaker_embedding(encoder_outputs, gst_outputs)
encoder_outputs.shape

In [ ]:
gst_outputs_att.shape

In [ ]:
gst_outputs_att

In [ ]:
_

In [ ]:
for k_token, v_amplifier in style_wav.items():
    print(type(v_amplifier))

In [ ]:
model.gst_layer.style_token_layer.style_tokens.shape

In [ ]:
model.gst_layer.style_token_layer.style_tokens[0,:]*0.3

In [ ]:
model.cuda()

In [82]:
style_mel = compute_style_mel(style_wav, ap, cuda=True)
style_mel = numpy_to_torch(style_mel, torch.float, cuda=use_cuda)

In [83]:
style_mel.shape

torch.Size([1, 80, 1206])

In [86]:
s = "ambulantes vendem os ingressos para a copa do mundo de dois mil e catorze , mas só alguns dias antes do dia do sorteio ."

# inputs = text_to_seqvec(s, CONFIG)
# inputs = numpy_to_torch(inputs, torch.long, cuda=use_cuda)
# inputs = inputs.unsqueeze(0)

# embedded_inputs = model.embedding(inputs).transpose(1, 2)
# encoder_outputs = model.encoder.inference(embedded_inputs)

# print(inputs.shape, encoder_outputs.shape )

encoder, logits = model.cuda().gst_layer(style_mel)

In [87]:
logits

tensor([[[[0.3521, 0.3196, 0.3283]]]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)

In [88]:
out = torch.cat(
            torch.split(logits, 1, dim=0),
            dim=3).squeeze(0)

In [90]:
out.shape

torch.Size([1, 1, 3])

In [91]:
logits.shape

torch.Size([1, 1, 1, 3])